<a href="https://colab.research.google.com/github/ayushnag/sophy/blob/main/db/sophy_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SOPHY Database
### Southern Ocean Phytoplankton Database
- Made by Ayush Nag and Hannah-Joy Warren
- UW Oceanography (add other credits)

In [1]:
3 + 7

10